<a href="https://colab.research.google.com/github/ledbagholberton/holbertonschool-machine_learning/blob/master/Copia_de_0_transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 43kB/s 
     |████████████████████████████████| 3.9MB 39.4MB/s 
     |████████████████████████████████| 450kB 50.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=d0c96ef93c8565998669a7ab602709e8db3e6d0257aac9a976cb99d286b2da5d
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0rc2 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0rc2 has requirement tensorflow-estimator<2.3.0,>=2.2.0rc0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflo

In [19]:
"""FUnction transfer
Write a python script that trains a convolutional neural network to
classify the CIFAR 10 dataset:

You must use one of the applications listed in Keras Applications
Your script must save your trained model in the current working directory
as cifar10.h5
Your saved model should be compiled
Your saved model should have a validation accuracy of 88% or higher
Your script should not run when the file is imported
Hint: The training may take a while, start early!
In the same file, write a function def preprocess_data(X, Y): that
pre-processes the data for your model:

X is a numpy.ndarray of shape (m, 32, 32, 3) containing the CIFAR 10 data,
where m is the number of data points
Y is a numpy.ndarray of shape (m,) containing the CIFAR 10 labels for X
Returns: X_p, Y_p
X_p is a numpy.ndarray containing the preprocessed X
Y_p is a numpy.ndarray containing the preprocessed Y
"""
import tensorflow.keras as K
import numpy as np


if __name__ == '__main__':
    (X, Y), _ = K.datasets.cifar10.load_data()
    X_train = K.applications.densenet.preprocess_input(X)
    y_train = K.utils.to_categorical(Y, 10)
    expand_x_train = np.fliplr(X_train)
    X_train = np.concatenate([X_train, expand_x_train])
    y_train = np.concatenate([y_train, y_train])
    model = K.applications.densenet.DenseNet121(
        weights='imagenet', 
        include_top=False,
        input_shape=(32,32,3),
        classes=10
    )
    """for layer in model.layers[:300]:
        layer.trainable = False
    for layer in model.layers[300:]:
        layer.trainable = True """
    out_0 = model.layers[-1].output
    drop_1 = K.layers.Dropout(0.4)(out_0)
    out_1 = K.layers.Flatten()(drop_1)
    out_2 = K.layers.Activation('relu')(out_1)
    dense_1 = K.layers.Dense(256, activation='relu')(out_2)
    drop_1 = K.layers.Dropout(0.4)(dense_1)
    dense_3 = K.layers.Dense(10, activation='softmax')(drop_1)
    model = K.models.Model(inputs=model.inputs, outputs=dense_3)
    for layer in model.layers[:80]:
        layer.trainable = False
    for layer in model.layers[81:]:
        layer.trainable = True
    model.summary()
    model.compile(
        loss='categorical_crossentropy',
        optimizer=K.optimizers.RMSprop(lr=1e-4),
        metrics=['accuracy']
    )
    lrr=K.callbacks.ReduceLROnPlateau(
                monitor='accuracy',
                factor=.01,
                patience=2,
                min_lr=1e-5)
    e_stop = K.callbacks.EarlyStopping(monitor='loss', patience=3)
    checkpoint = K.callbacks.ModelCheckpoint(filepath='./mymodel', monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    nb = 32
    epochs = 15
    hist = model.fit(
        x=X_train,
        y=y_train,
        batch_size=nb,
        epochs=epochs,
        verbose=1,
        callbacks=[lrr, e_stop, checkpoint]
    )
    model.save('cifar10.h5')

def preprocess_data(X, Y):
    """Function preprocess data """
    import tensorflow.keras as K

    x_p = K.applications.densenet.preprocess_input(X)
    y_p = K.utils.to_categorical(Y, 10)
    return (x_p, y_p)


Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
zero_padding2d_16 (ZeroPadding2 (None, 38, 38, 3)    0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 16, 16, 64)   9408        zero_padding2d_16[0][0]          
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1/conv[0][0]                 
___________________________________________________________________________________________

In [20]:
"""Main"""

import tensorflow.keras as K

# fix issue with saving keras applications
K.learning_phase = K.backend.learning_phase 

_, (X, Y) = K.datasets.cifar10.load_data()
X_p = K.applications.densenet.preprocess_input(X)
Y_p = K.utils.to_categorical(Y, 10)
model = K.models.load_model('cifar10.h5')
a = model.evaluate(X_p, Y_p, batch_size=128, verbose=1)
print(a)

79/79 [==============================] - 2s 24ms/step - loss: 1.4301 - accuracy: 0.8213
[1.430068016052246, 0.8213000297546387]
